In the last module, we processed RADseq data from the red diamond rattlesnake (Crotalus ruber) to assemble it from raw reads into aligned data. We now have files that are sequences of SNPs (single nucleotide polymorphisms) or loci that are on a common coordinate system for each individual sample. This allows us to make inferences based on variation at sites in the alignment across samples. We'll do some basic population genetic analyses to look at population structure. We'll run all of this in R.



In [ ]:
# necessary for me on MedicineBow cluster
.libPaths(c("/cluster/medbow/home/sharrin2/R/x86_64-pc-linux-gnu-library/4.4",
            "/apps/u/spack/gcc/14.2.0/r/4.4.0-w7xoohc/rlib/R/library"))

In [ ]:
# uncomment for actual tutorial, commented for dev
# install.packages(c("adegenet", "plotrix", "mapdata", "BiocManager", "vcfR", "fossil", "scatterpie", "mapproj", "MASS"))
# BiocManager::install("LEA")

This may take several minutes. Once that's done, we can load up the packages:

In [ ]:
# Load up necessary packages
library(vcfR)
library(adegenet)
library(LEA)
library(mapdata)
library(ggplot2)
library(scatterpie)
library(mapproj)
library(fossil)
library(MASS)

## tweak for gcp, obviously

Then we will specify a number of file paths and read in a few files so that we don’t have to repeatedly hardcode file paths farther down in the notebook. This makes it easier to reuse the script on different datasets or the same data with different filtering schemes without having to search through the script for every time an absolute file path is specified.

In [ ]:
## Set up an object containing the path to the data
data_dir <- "/home/sharrin2/radseq_cloud/ruber_reduced_denovo_outfiles"

## make a directory to put the output plots into
##    this can be wherever you like, I'm putting it into the directory that contains
##    my scripts and ipyrad output directories
out_dir <- "/home/sharrin2/radseq_cloud/pop_struct_out"
if(!dir.exists(out_dir)){ # check if the directory  exists and return TRUE if it does not
  dir.create(out_dir)   # create the directory if it does not exist
}

# Set up an object that contains the base file name of files in the output directory. 
#    Data files are all this basename with varying extensions
#  we won't call this 'basename' because that is a function in R
#  setting up the files this way allows us to easily run this script on another assembly 
#    without needing to edit file names everywhere they occur
basefile <- "ruber_reduced_denovo"

# Read in the geographic coordinates for plotting later
coords <- read.csv("/home/sharrin2/radseq_cloud/ruber_data/Localities.csv", header=TRUE, row.names=NULL)



####################################################################################
## Set up paths to input files using the base file name specified above
####################################################################################
path_ugeno <- paste0(data_dir,"/", basefile,".ugeno")
path_ustr <- paste0(data_dir,"/", basefile,".ustr")
path_vcf <- paste0(data_dir,"/", basefile,".vcf")

### Set up some colors for plotting farther down
colors_2 <- c("red", "blue") # colors for plotting 2 populations


# Set the working directory to the output directory
setwd(out_dir) # set directory as the output directory

## Basic stats

Let's start by reading in the data and calculating some very basic population genetic summary statistics.

Read in the dat a from the vcf file:

In [ ]:
gendata_all <- read.vcfR(path_vcf) # read in all of the genetic data from the vcf file
gendata <- vcfR2genind(gendata_all) # convert to genind format

Calculate observed and expected heterozygosity:


In [ ]:
# Use adegenet's summary function on the genind object
genstats <- summary(gendata)

# and make overlapping histograms
hist(genstats$Hobs, breaks = 15)
hist(genstats$Hexp, add = TRUE, col = scales::alpha('blue',.25))

Looks like we might have lower observed than expected heterozygosity. Let's check the means across loci to confirm:

In [ ]:
# and look at the mean of each
mean(genstats$Hobs)
mean(genstats$Hexp)

Yep, we have low observed heterozygosity compared to expected. One likely reason for this is if there is population structure in the data. We'll test for this shortly.